# SLTimer Tutorial 
This notebook shows you how to find the estimatation of a lens time delays from example light curve data using the `PyCS` code. For a detailed tutorial through the `PyCS` code, please visit this [address.](https://github.com/DarkEnergyScienceCollaboration/SLTimer/blob/master/notebooks/PyCS_Tutorial.ipynb)

Let's first import `SLTimer`, as well as a few other important commands. 

In [ ]:
from __future__ import print_function
%matplotlib inline
from desc.sltimer import SLTimer
%load_ext autoreload
%autoreload 2

Next, let's simplify our variable. 

In [ ]:
S = SLTimer()

Let's define our variables and download our trial data, testing it to make sure that we have downloaded the correct file. The output should state 194 lines if using the trialcurves.txt file. 

In [ ]:
url = "https://raw.githubusercontent.com/COSMOGRAIL/PyCS/master/demo/demo1/data/trialcurves.txt"
S.download(url)

These lines will run an entire free-knot spline technique on your data with a complete error analysis using the `PyCS` method.

In [ ]:
S.create_spline_model()
S.delete_old_files()
S.estimate_uncertainties(n=1,npkl=4)

This is how you can specify how the time delays will be analyzed. The default is listed below according to the `PyCS` tutorial. See the bottom of the page for alternate methods. 

In [ ]:
S.estimate_time_delays(method='pycs',microlensing='polynomial',agn='spline',error=None)

To compare the time delays between the basic method(no microlensing),the polynomial method with microlensing, and the free-knot spline method, run these lines.

In [ ]:
S.basic_time_delays()
S.polynomial_time_delays()
S.spline_time_delays()

To compare the models between the basic method(no microlensing),the polynomial method with microlensing, and the free-knot spline method, run these lines.

In [ ]:
S.create_basic_model()
S.create_polynomial_model()
S.create_spline_model()

To display the sample light curves from the PyCS demo tutorial. 

In [ ]:
S.display_light_curves()

## Alternate Methods of Estimating Time Delays:
### Error:
By default, `error=None`:
>To find the intrinsic variance: `error='intrinsic variance'`.

>To find the error bars, the relationship between the points, and the instrinsic variance: `error='complete'`
